In [30]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
from torch import nn
from torch.autograd import Variable

In [31]:
from MachineLearning import get_features, create_heatmap, get_images
from FeatureExtractor import eval_images, get_layer_3_features

feats = eval_images('/Users/will/StreetViewProject/streetview_images/33960272_-83296329', 'final')
# eval_images('/Users/will/StreetViewProject/streetview_images/33960272_-83296329', 'final')

# imgFolder = '/Users/will/StreetViewProject/streetview_images/3392401699999999_-83259033'
# imgs, showableImages = get_images(imgFolder)
# feats, imgs = get_features(imgs)



tensor([0.0136, 0.0559, 0.1135,  ..., 0.0075, 0.2303, 0.1790],
       grad_fn=<SelectBackward0>)
torch.Size([2048])
tensor([0.0614, 0.0026, 0.0857,  ..., 0.1029, 0.0000, 0.0150],
       grad_fn=<SelectBackward0>)
torch.Size([2048])
tensor([0.0675, 0.2197, 0.0712,  ..., 0.0053, 0.0585, 0.0240],
       grad_fn=<SelectBackward0>)
torch.Size([2048])
tensor([0.0184, 0.1379, 0.0447,  ..., 0.2087, 0.0261, 0.2257],
       grad_fn=<SelectBackward0>)
torch.Size([2048])


In [33]:
feats = torch.cat(feats, 0)
# feats.shape

In [34]:
labels = torch.FloatTensor([[80.0], [1.0], [30.0], [20.0]])


In [27]:
labels.shape

torch.Size([4, 1])

In [35]:
class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)  

    def forward(self, x):
        out = self.linear(x)
        return out

In [42]:
model = LinearRegressionModel(2048, 1)
# model.hidden = model.init_hidden()

In [43]:
criterion = nn.MSELoss()
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [46]:
epochs = 3
losses = []
for epoch in range(epochs):
    epoch += 1
    # Convert numpy array to torch Variable
    # inputs = torch.from_numpy(x_train).requires_grad_()
    inputs = feats
    labels = labels

    # print(inputs)

    # Forward to get output
    outputs = model(inputs[0])

    # Calculate Loss
    loss = criterion(outputs, labels)
    losses.append(loss)

    # Clear gradients w.r.t. parameters
    optimizer.zero_grad()

    # Getting gradients w.r.t. parameters
    loss.backward(retain_graph=True)

    # Updating parameters
    optimizer.step()

    print('epoch {}, loss {}'.format(epoch, loss.item()))


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [278]:
from torchvision.models import ResNet50_Weights, resnet50

weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights)
model.layer4 = torch.nn.Identity()
model.avgpool = torch.nn.Identity()
model.fc = torch.nn.Identity()
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 